In [1]:
import talib
import numpy as np
import pandas as pd
import random
from collections import deque

In [2]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M15_2019-2020.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

df_small = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2019-2020.csv')
df_small.Date = pd.to_datetime(df_small.Date)
df_small.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2019-07-30 07:00:00   1.21460   1.21575  1.21387    1.21557   1.21480   
1 2019-07-30 07:15:00   1.21560   1.21697  1.21559    1.21662   1.21577   
2 2019-07-30 07:30:00   1.21666   1.21796  1.21666    1.21785   1.21682   
3 2019-07-30 07:45:00   1.21780   1.21780  1.21623    1.21637   1.21798   
4 2019-07-30 08:00:00   1.21641   1.21655  1.21596    1.21638   1.21657   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.21591  1.21404    1.21574   1.21470   1.21582  1.21396    1.21566  
1   1.21714  1.21577    1.21679   1.21568   1.21705  1.21568    1.21670  
2   1.21812  1.21682    1.21800   1.21674   1.21803  1.21674    1.21792  
3   1.21798  1.21640    1.21653   1.21789   1.21789  1.21632    1.21645  
4   1.21672  1.21612    1.21654   1.21649   1.21664  1.21604    1.21646  
---------------------------------------------------------------------------
                     Date  Bi

In [5]:
print(df_small.head())
print('-' * 75)
print(df_small.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2019-07-30 07:00:00   1.21460   1.21497  1.21387    1.21447   1.21480   
1 2019-07-30 07:05:00   1.21449   1.21490  1.21449    1.21476   1.21468   
2 2019-07-30 07:10:00   1.21478   1.21575  1.21477    1.21557   1.21495   
3 2019-07-30 07:15:00   1.21560   1.21693  1.21559    1.21679   1.21577   
4 2019-07-30 07:20:00   1.21675   1.21697  1.21647    1.21665   1.21693   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.21513  1.21404    1.21464   1.21470   1.21505  1.21396    1.21456  
1   1.21512  1.21468    1.21490   1.21458   1.21500  1.21458    1.21483  
2   1.21591  1.21495    1.21574   1.21486   1.21582  1.21486    1.21566  
3   1.21712  1.21577    1.21695   1.21568   1.21702  1.21568    1.21687  
4   1.21714  1.21663    1.21681   1.21684   1.21705  1.21656    1.21673  
---------------------------------------------------------------------------
                     Date  Bi

In [6]:
def add_fractal(df, i, look_back=3):
  if i >= look_back and i < df.shape[0] - look_back:
    lows = []
    highs = []

    for j in range(1, look_back + 1):
      prev_bid_low, prev_bid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
      future_bid_low, future_bid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

      lows.append(float(prev_bid_low))
      lows.append(float(future_bid_low))
      highs.append(float(prev_bid_high))
      highs.append(float(future_bid_high))

    bid_low, bid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

    if float(bid_low) < min(lows):
      return 1

    elif float(bid_high) > max(highs):
      return 2

    else:
      return 0

  else:
    return np.nan


def add_beep_boop(row):
  macdhist, ema50, mid_low, mid_high = row[['macdhist', 'ema50', 'Mid_Low', 'Mid_High']]

  if float(macdhist) > 0 and float(mid_low) > float(ema50):
    return 1

  elif float(macdhist) < 0 and float(mid_high) < float(ema50):
    return 2

  else:
    return 0

In [7]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['ema50'] = talib.EMA(df['Mid_Close'], timeperiod=50)
df['ema30'] = talib.EMA(df['Mid_Close'], timeperiod=30)
df['ema25'] = talib.EMA(df['Mid_Close'], timeperiod=25)
df['ema20'] = talib.EMA(df['Mid_Close'], timeperiod=20)
df['ema15'] = talib.EMA(df['Mid_Close'], timeperiod=15)
df['atr'] = talib.ATR(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=500)
# df['fractal'] = [add_fractal(df, i) for i in range(df.shape[0])]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df['beep_boop'] = df.apply(add_beep_boop, axis=1)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df['beep_boop'].value_counts())
# print(df['fractal'].value_counts())

In [ ]:
value_per_pip = 1.0
max_pips_to_risk = 0.0100
amounts_per_day = [-4, -5, -6]

In [ ]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_simulation(risk_reward_ratio, pullback_cushion, atr_percentage, last_x_bars, fractal_distance, bar_length, max_open_trades=1):
    pullback_cushion /= 10000
    fractal_distance /= 10000
    bar_length /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trades = deque(maxlen=max_open_trades)

    def iterate_thru_small_data(trade):
        def get_end_date(j):
          return df_small.loc[df_small.index[j + 1], 'Date'] if j + 1 < len(df_small) else None

        j = df_small[df_small.Date >= trade['start_date']].index[0]

        nonlocal reward
        nonlocal day_fees
        nonlocal n_losses
        nonlocal n_wins
        nonlocal curr_win_streak
        nonlocal curr_loss_streak
        nonlocal win_streak
        nonlocal loss_streak

        while j < len(df_small):
          curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_date = \
          df_small.loc[df_small.index[j], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low',
                                         'Ask_Close', 'Date']]

          if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
              trade['stop_loss'] = curr_bid_high - trade['pips_risked']

          if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
            trade['stop_loss'] = trade['pips_risked'] + curr_ask_low

          if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          j += 1


    i = 3

    while i < len(df):
        if i < last_x_bars:
            i += 1
            continue

        curr_date = df.loc[df.index[i], 'Date']

        # Remove any expired trades -----------------------------------------
        trades_to_remove = []

        for trade in trades:
          if trade['end_date'] is not None and trade['end_date'] >= curr_date:
            trades_to_remove.append(trade)

        for trade in trades_to_remove:
          trades.remove(trade)
        # -------------------------------------------------------------------

        ema200 = df.loc[df.index[i - 1], 'ema200']
        ema30 = df.loc[df.index[i - 1], 'ema30']
        ema25 = df.loc[df.index[i - 1], 'ema25']
        ema20 = df.loc[df.index[i - 1], 'ema20']
        ema15 = df.loc[df.index[i - 1], 'ema15']
        atr = df.loc[df.index[i - 1], 'atr']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        enough_volatility = (spread / atr) <= atr_percentage
        curr_al = df.loc[df.index[i - 1], 'Ask_Low']
        curr_bh = df.loc[df.index[i - 1], 'Bid_High']
        previous_ask_lows = []
        previous_bid_highs = []
        curr_ml = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh - curr_ml)
        beep_boop1 = df.loc[df.index[i - 1], 'beep_boop']
        beep_boop2 = df.loc[df.index[i - 2], 'beep_boop']
        beep_boop3 = df.loc[df.index[i - 3], 'beep_boop']

        for ind in range(i - last_x_bars, i):
          previous_ask_lows.append(df.loc[df.index[ind], 'Ask_Low'])
          previous_bid_highs.append(df.loc[df.index[ind], 'Bid_High'])

        process_trade = False

        if beep_boop3 == 1 and beep_boop2 == 1 and beep_boop1 == 1 and curr_ml > ema200 and enough_volatility and len(trades) < max_open_trades and curr_al <= min(previous_ask_lows) and curr_bar_length <= bar_length:
            all_buys = True

            for trade in trades:
                if trade['trade_type'] != 'buy':
                    all_buys = False
                    break

            if all_buys:
              pullback = ema20
              # pullback = None
              # u = i - 4

              # while u >= 0:
              #     if df.loc[df.index[u], 'fractal'] == 1:
              #         curr_pullback = float(df.loc[df.index[u], 'Bid_Low'])
              #         curr_fractal_distance = curr_ao - curr_pullback

              #         if curr_fractal_distance >= fractal_distance:
              #             pullback = curr_pullback
              #             break

              #     u -= 1

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_ask_open)
                  stop_loss = round(pullback - pullback_cushion, 5)
                  # stop_loss = round(pullback - pullback_cushion, 3)

                  if stop_loss < open_price:
                      curr_pips_to_risk = open_price - stop_loss

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                            'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                            'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_buys += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        elif beep_boop3 == 2 and beep_boop2 == 2 and beep_boop1 == 2 and curr_mh < ema200 and enough_volatility and len(trades) < max_open_trades and curr_bh >= max(previous_bid_highs) and curr_bar_length <= bar_length:
            all_sells = True

            for trade in trades:
                if trade['trade_type'] != 'sell':
                    all_sells = False
                    break

            if all_sells:
              pullback = ema20
              # pullback = None
              # u = i - 4

              # while u >= 0:
              #     if df.loc[df.index[u], 'fractal'] == 2:
              #         curr_pullback = float(df.loc[df.index[u], 'Ask_High'])
              #         curr_fractal_distance = curr_pullback - curr_bo

              #         if curr_fractal_distance >= fractal_distance:
              #             pullback = curr_pullback
              #             break

              #     u -= 1

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_bid_open)
                  stop_loss = round(pullback + pullback_cushion, 5)
                  # stop_loss = round(pullback + pullback_cushion, 3)

                  if stop_loss > open_price:
                      curr_pips_to_risk = stop_loss - open_price

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                  'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                  'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_sells += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        if process_trade:
          iterate_thru_small_data(trades[-1])

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak

In [ ]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.5, 1.6, 1.7, 1.8, 1.9, 2]
# risk_reward_ratio_vals = [2]
pullback_cushion_vals = [0, 5, 10]
# pullback_cushion_vals = [10]
atr_percentages = [0.20, 0.25, 0.30]
# atr_percentages = [0.25]
last_x_bars_vals = [2, 3]
# last_x_bars_vals = [2]
# fractal_distances = [0, 5, 10]
fractal_distances = [10]
bar_lengths = [5, 10]
# bar_lengths = [5]
n_possibilities = len(risk_reward_ratio_vals) * len(pullback_cushion_vals) * len(atr_percentages) * len(last_x_bars_vals) * len(fractal_distances) * len(bar_lengths)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
  for pullback_cushion in pullback_cushion_vals:
    for atr_percentage in atr_percentages:
      for last_x_bars in last_x_bars_vals:
        for fractal_distance in fractal_distances:
          for bar_length in bar_lengths:
            all_combos.append((risk_reward_ratio, pullback_cushion, atr_percentage, last_x_bars, fractal_distance, bar_length))

# percentage_to_try = 0.75
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_pullback_cushion = None
best_atr_percentage = None
best_last_x_bars_val = None
best_fractal_distance = None
best_bar_length = None
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, pullback_cushion, atr_percentage, last_x_bars, fractal_distance, bar_length in combos_to_try:
  reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak = run_simulation(risk_reward_ratio, pullback_cushion, atr_percentage, last_x_bars, fractal_distance, bar_length)
  runs_finished += 1

  print(reward)
  print('Num buys: ' + str(n_sells))
  print('Num sells: ' + str(n_buys))
  print('Num trades: ' + str(n_buys + n_sells))
  print('Num wins: ' + str(n_wins))
  print('Num losses: ' + str(n_losses))
  print('Win streak: ' + str(win_streak))
  print('Loss streak: ' + str(loss_streak))
  print('Remaining runs: ' + str(n_runs - runs_finished))

  if reward > best_reward:
    best_reward = reward
    best_risk_reward = risk_reward_ratio
    best_pullback_cushion = pullback_cushion
    best_atr_percentage = atr_percentage
    best_last_x_bars_val = last_x_bars
    best_fractal_distance = fractal_distance
    best_bar_length = bar_length

  print('Best reward so far: ' + str(best_reward))
  print()

Num runs: 216

-60.50000000000777
Num buys: 2
Num sells: 1
Num trades: 3
Num wins: 0
Num losses: 3
Win streak: 0
Loss streak: 3
Remaining runs: 215
Best reward so far: -60.50000000000777

-4276.4999999998
Num buys: 207
Num sells: 176
Num trades: 383
Num wins: 120
Num losses: 263
Win streak: 8
Loss streak: 21
Remaining runs: 214
Best reward so far: -60.50000000000777

-165.00000000002714
Num buys: 34
Num sells: 19
Num trades: 53
Num wins: 14
Num losses: 39
Win streak: 5
Loss streak: 18
Remaining runs: 213
Best reward so far: -60.50000000000777

-13.999999999991797
Num buys: 1
Num sells: 0
Num trades: 1
Num wins: 0
Num losses: 1
Win streak: 0
Loss streak: 1
Remaining runs: 212
Best reward so far: -13.999999999991797

-111.49999999993048
Num buys: 52
Num sells: 38
Num trades: 90
Num wins: 28
Num losses: 62
Win streak: 7
Loss streak: 9
Remaining runs: 211
Best reward so far: -13.999999999991797

-3941.9999999999336
Num buys: 87
Num sells: 63
Num trades: 150
Num wins: 41
Num losses: 109
Win

In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best pullback cushion: ' + str(best_pullback_cushion))
print('Best atr percentage: ' + str(best_atr_percentage))
print('Best last x bars: ' + str(best_last_x_bars_val))
print('Best fractal distance: ' + str(best_fractal_distance))
print('Best bar length: ' + str(best_bar_length))

# 2014-2015:
    # EMA 20:
    # Best reward: 1712.499999999993
    # Best risk/reward ratio: 1.8
    # Best pullback cushion: 10
    # Best atr percentage: 0.25
    # Best last x bars: 2
    # Best fractal distance: 10
    # Best bar length: 10
    

# 2015-2016:
    # EMA 20:
    # Best reward: 1556.9999999999332
    # Best risk/reward ratio: 1.6
    # Best pullback cushion: 10
    # Best atr percentage: 0.25
    # Best last x bars: 2
    # Best fractal distance: 10
    # Best bar length: 10


# 2016-2017:
    # EMA 20:
    # Best reward: 2429.0000000000427
    # Best risk/reward ratio: 1.7
    # Best pullback cushion: 10
    # Best atr percentage: 0.2
    # Best last x bars: 2
    # Best fractal distance: 10
    # Best bar length: 10


# 2017-2018:
    # EMA 20:
    # Best reward: 799.0000000000182
    # Best risk/reward ratio: 2
    # Best pullback cushion: 10
    # Best atr percentage: 0.2
    # Best last x bars: 3
    # Best fractal distance: 10
    # Best bar length: 5


# 2018-2019:
    # EMA 20:
    # Best reward: 994.5000000000528
    # Best risk/reward ratio: 1.8
    # Best pullback cushion: 10
    # Best atr percentage: 0.25
    # Best last x bars: 3
    # Best fractal distance: 10
    # Best bar length: 10


# 2019-2020:


# 2020-2021:


------------ FINAL RESULTS ------------
Best reward: 994.5000000000528
Best risk/reward ratio: 1.8
Best pullback cushion: 10
Best atr percentage: 0.25
Best last x bars: 3
Best fractal distance: 10
Best bar length: 10
